In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2520)

In [15]:
x[:,:2]

matrix([[  4054.27745605,   3357.3984904 ],
        [  4325.51681309,   4678.20980891],
        [  7033.59877281,   6453.67986055],
        [  5559.94078391,   6512.51029391],
        [  8993.26083982,   6470.39530625],
        [  5764.77301069,   5313.77310057],
        [  5718.38069914,   4333.41422322],
        [  4358.45760101,   5367.74844054],
        [  6090.65904116,   8051.29662464],
        [  5590.52387593,   5879.329336  ],
        [ 10103.59423016,   7219.81802949],
        [  6712.57890522,   9235.92014542],
        [  5549.75848202,   3445.95657912],
        [  4936.30363816,   2998.73689879],
        [  2607.53454639,   4553.68429623],
        [  3511.46242948,   1619.19983651],
        [  5487.88656664,   5627.53659871],
        [  3856.26600126,   5116.61759177],
        [  9129.10897488,  10057.80926054],
        [  4968.48273056,   5006.21682966],
        [ 10626.27438914,   5557.25162458],
        [  5629.17649226,   4852.06087378],
        [  5642.98651953,   3052

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [9e-02, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.85128387e+06  0.00000000e+00  8.06e+03 4.37e+02  8.37e+05     0s
   1   1.78545337e+06 -4.15929640e+06  2.30e+02 2.44e+01  1.33e+05     0s
   2  -7.21700940e+04 -1.11633499e+06  2.30e-04 1.14e-13  1.86e+04     0s
   3  -3.41323251e+05 -4.68023052e+05  3.29e-06 2.84e-14  2.2

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[8.546493573131865e-10,
 9049.049299622602,
 9.816834008165531e-10,
 5.52610338104443e-07,
 2058.740891625823,
 9.227459005934632e-07,
 2.2408451228153092e-07,
 2469.2221396911605,
 1.0737853121266246e-09,
 5265.199257294281,
 1.07170252675216e-09,
 1.073781694478912e-09,
 1.0725384127739955e-09,
 1.0786798346086824e-09,
 5494.952967978289,
 1800.5806899942388,
 5.490576195496414e-07,
 5.534319943040927e-07,
 2473.363445983496,
 10838.2687893305,
 9.202918785295598e-07,
 0.00015879023873487629,
 4901.1105742916925,
 3.960858749725616e-06,
 5005.889329206605,
 70.30090538668935,
 75.82210230779071,
 4793.371468253723,
 3.95632011757628e-06,
 3.8952241674603705e-06,
 3.954383916281303e-06,
 3506.3958147840162,
 5161.53900853867,
 3747.7332202487682,
 1375.2457432075023,
 2471.6158868550033,
 1386.1958669281921,
 1.9083638965420213e-05,
 808.7350315798872,
 3138.518104694666,
 3032.528825679184,
 968.9309904952413,
 7.35563790237029e-06,
 8.114332951124066e-06,
 13502.995350387382,
 37.41